##  Probit models to forecast binary outcomes such as recessions
<div style="text-align: right"> Fogli Alessandro </div>
<div style="text-align: right"> ID 231273 </div>
<div style="text-align: right"> Project #3 </div>

### Install packages

In [14]:
import scipy.stats as si
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import pandas_datareader as pdr
import datetime
from IPython.display import display, HTML
import datetime as dt
import getFamaFrenchFactors as gff
import quandl
from fredapi import Fred
import config
fred = Fred(api_key= config.fred_api)
QUANDL_KEY = config.quandl_key
quandl.ApiConfig.api_key = QUANDL_KEY
from scipy import stats
from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.discrete.discrete_model import Probit
import statsmodels.formula.api as smf

### Data (all quarterly)

Get data of difference in yields between 10 year and 3 months U.S. Treasuries

In [2]:
t10y = fred.get_series('DGS10', observation_start="1962-01-02",observation_end= "2021-12-31",frequency='q')
t3mo = fred.get_series('TB3MS', observation_start="1962-01-02",observation_end= "2021-12-31",frequency='q')
term = t10y-t3mo
term = term.dropna()
#term = term.tolist()
term = term.to_frame().rename(columns={0: 'term'})
#240

Get data of Federal Funds Rate

In [3]:
funds_rate = fred.get_series('FEDFUNDS', observation_start="1962-01-02", observation_end= "2021-12-31" ,frequency='q')
funds_rate = funds_rate.tolist()
#240

Get data of GDP Growth

In [4]:
gdp = fred.get_series('GDP', observation_start="1962-01-02", observation_end= "2021-12-31",frequency='q', units='pch')
gdp = gdp.tolist()
#240 miss first quarter 2022

NBER based Recession Indicators for the United States

In [3]:
nber = fred.get_series('USREC', observation_start="1962-01-01", observation_end= "2021-12-31" ,frequency='q')
nber = nber.astype(int)
#nber = nber.tolist()
#term.isna().sum().sum()
#count = np.isinf(term).values.sum()
nber = nber.to_frame().rename(columns={0: 'nber'})
nber_plot = nber.reset_index().rename(columns={'index': 'date'})

Load data of S&P500 Futures - Jun 2022 expiring date

In [6]:
sp500_futures = pd.read_csv('S&P 500 Futures Historical Data.csv', index_col = 0)
sp500_futures.index = pd.to_datetime(sp500_futures.index, format= '%b %y') # date index transforming
sp500_futures.index = sp500_futures.index + pd.offsets.MonthEnd()
sp500_futures_rtn = sp500_futures['Change %']
sp500_futures_rtn= sp500_futures_rtn.str.replace('%','') # remove str % from value
sp500_futures_rtn = pd.to_numeric(sp500_futures_rtn, errors='coerce')
sp500_futures_rtn.fillna(0, inplace=True)
sp500_futures_rtn = sp500_futures_rtn.iloc[::-1]
sp500_futures_rtn.drop(sp500_futures_rtn.tail(1).index,inplace=True) # adjusting size to match stocks month

Get data of S&P500

In [7]:
'''
sp500 = quandl.get("MULTPL/SP500_REAL_PRICE_MONTH", start_date='1961-10-01', end_date='2021-12-31', collapse="monthly")
sp500_quarter_rtn = sp500.resample("3M").mean()
sp500_quarter_rtn = sp500_quarter_rtn.pct_change()
sp500_quarter_rtn = sp500_quarter_rtn.dropna()
sp500_quarter_rtn.drop(sp500_quarter_rtn.tail(1).index,inplace=True)
sp500_quarter_rtn = sp500_quarter_rtn.apply(lambda x: x* 100)
'''

'\nsp500 = quandl.get("MULTPL/SP500_REAL_PRICE_MONTH", start_date=\'1961-10-01\', end_date=\'2021-12-31\', collapse="monthly")\nsp500_quarter_rtn = sp500.resample("3M").mean()\nsp500_quarter_rtn = sp500_quarter_rtn.pct_change()\nsp500_quarter_rtn = sp500_quarter_rtn.dropna()\nsp500_quarter_rtn.drop(sp500_quarter_rtn.tail(1).index,inplace=True)\nsp500_quarter_rtn = sp500_quarter_rtn.apply(lambda x: x* 100)\n'

### Probit model

A probit regression is a version of the generalized linear model used to model dichotomous outcome variables. It uses the inverse standard normal distribution as a linear combination of the predictors. The binary outcome variable Y is assumed to have a Bernoulli distribution with parameter p (where the success probability is p∈(0,1). Hence, the probit link function is:
$$ probit(Y) = \sum_{k=0}^n \beta_{k} x_{ik} $$

The Probit model assumes that the firm’s probability of recession has a cumulative standard-normal distribution, rather than a logistic distribution. However, by multiplying the results of the logistic distribution by an appropriate coefficient the distribution of the Probit model can be obtained.

Implementing Probit model with only Term as explanatory variable

In [6]:
term_nber = term.join(nber['nber'])

In [14]:
term = term.shift(1).iloc[1: , :]
term.isnull().values.any()
count = np.isinf(term).values.sum()
print(count)

0


In [16]:
nber = nber.iloc[1: , :]
nber.isnull().values.any()
count = np.isinf(term).values.sum()
print(count)

0


In [21]:
def fill(): #valx = factors, valy = nber
    term_nber = term.join(nber['nber'])

    results = pd.DataFrame({ 
                    "Quarter ahead": [],
                    "observations": [],
                    "Beta": [],
                    "Alpha": [],
                    "R-sq": [],
                    
                })
    for i in range(1,13):
        term_nber['term'].shift(-i) #switch lag and drop first val (Nan) of independent variable
        term_nber = term_nber.iloc[: -1 , :] #drop firs row of dependent var
        X =  term_nber['term'] #term
        y =  term_nber['nber'] #nber
        X = sm.add_constant(X)
        model = Probit(y, X).fit()
        observation = len(term_nber)
        beta = model.params['term']
        alpha = model.params['const']
        rsq = model.prsquared
        ahead = i

        results.loc[i-1] = pd.Series({'Quarter ahead': ahead, 'observations': observation, 'Beta': beta,'Alpha': alpha, 'R-sq': rsq})
        
    results.set_index('Quarter ahead', inplace=True)
    display(HTML(results.to_html(classes='table table-stripde')))
fill()

Optimization terminated successfully.
         Current function value: 0.369346
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.370347
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.371363
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.372377
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.373380
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.374381
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.375386
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.376384
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.377382
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.378364
  

,observations,Beta,Alpha,R-sq
Quarter ahead,,,,
1.0,239.0,-0.029568,-1.124671,0.000644
2.0,238.0,-0.029912,-1.121620,0.000661
3.0,237.0,-0.029741,-1.119337,0.000656
4.0,236.0,-0.030051,-1.116309,0.000672
5.0,235.0,-0.031323,-1.111800,0.000731
6.0,234.0,-0.033044,-1.106582,0.000814
7.0,233.0,-0.034761,-1.101348,0.000902
8.0,232.0,-0.037074,-1.095171,0.001025
9.0,231.0,-0.039518,-1.088760,0.001164


In [22]:
def fill(): #valx = factors, valy = nber
    term_nber = term.join(nber['nber'])

    results = pd.DataFrame({ 
                    "Quarter ahead": [],
                    "observations": [],
                    "Beta": [],
                    "Alpha": [],
                    "R-sq": [],
                    
                })
    for i in range(1,13):
        term_nber['term'].shift(i) #switch lag and drop first val (Nan) of independent variable
        term_nber = term_nber.iloc[1: , :] #drop firs row of dependent var
        X =  term_nber['term'] #term
        y =  term_nber['nber'] #nber
        X = sm.add_constant(X)
        model = Probit(y, X).fit()
        observation = len(term_nber)
        beta = model.params['term']
        alpha = model.params['const']
        rsq = model.prsquared
        ahead = i

        results.loc[i-1] = pd.Series({'Quarter ahead': ahead, 'observations': observation, 'Beta': beta,'Alpha': alpha, 'R-sq': rsq})
        
    results.set_index('Quarter ahead', inplace=True)
    display(HTML(results.to_html(classes='table table-stripde')))
fill()

Optimization terminated successfully.
         Current function value: 0.369341
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.370339
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.371342
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.372349
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.373358
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.374374
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.375386
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.376398
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.377416
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.378441
  

,observations,Beta,Alpha,R-sq
Quarter ahead,,,,
1.0,239.0,-0.029887,-1.124185,0.000658
2.0,238.0,-0.030430,-1.120829,0.000684
3.0,237.0,-0.030994,-1.117427,0.000712
4.0,236.0,-0.031670,-1.113838,0.000745
5.0,235.0,-0.032554,-1.109916,0.000789
6.0,234.0,-0.033367,-1.106084,0.000832
7.0,233.0,-0.034714,-1.101416,0.000902
8.0,232.0,-0.036304,-1.096353,0.000987
9.0,231.0,-0.037848,-1.091338,0.001075


In [16]:
term_nber = term.join(nber['nber'])
ff_model = smf.probit(
    formula='nber ~ term',
    data=term_nber).fit()
ff_model.summary()

Optimization terminated successfully.
         Current function value: 0.368345
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                   nber   No. Observations:                  240
Model:                         Probit   Df Residuals:                      238
Method:                           MLE   Df Model:                            1
Date:                Fri, 08 Apr 2022   Pseudo R-squ.:               0.0006437
Time:                        19:19:06   Log-Likelihood:                -88.403
converged:                       True   LL-Null:                       -88.460
Covariance Type:            nonrobust   LLR p-value:                    0.7358
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.1271      0.166     -6.806      0.000      -1.452      -0.803
term          -0.0296      0.088     -0.337      0.736      -0.202       0.142
==============================================================================
"""